In [7]:
import librosa 
from Praat import Praat_IdeaLogic as PLogic
from PIL import Image
import os
import glob
import joblib
import numpy as np
from tqdm.notebook import tqdm

##### Praat_IdeaLogic documentation: https://ml.qwe.re/notebooks/Wiktor/Audio%20Processing/Praat/Praat%20Custom%20Lib%20Documentation.ipynb

In [2]:
PLogic.get_sound_report("Audio_Data/Actor_01/03-01-01-01-01-01-01.wav")

From 0 to 0 seconds (duration Median pitch   Mean pitch  \
values only                       3.30329      108.562      111.849   
meta                    3.303292 seconds)   108.562 Hz   111.849 Hz   

            Standard deviation Minimum pitch Maximum pitch Number of pulses  \
values only             17.815        80.695       157.814              114   
meta                 17.815 Hz     80.695 Hz    157.814 Hz              114   

            Number of periods           Mean period  \
values only               109            0.00901355   
meta                      109   9.013554E-3 seconds   

            Standard deviation of period  ... Jitter (ddp) Shimmer (local)  \
values only                   0.00155166  ...        6.321          13.783   
meta                 1.551659E-3 seconds  ...       6.321%         13.783%   

            Shimmer (local, dB) Shimmer (apq3) Shimmer (apq5) Shimmer (apq11)  \
values only               1.169          5.228          8.955          13.764   
meta                   1.169 dB         5.228%         8.955%         13.764%   

            Shimmer (dda) Mean autocorrelation Mean noise-to-harmonics ratio  \
values only        15.684             0.757401                      0.390678   
meta              15.684%             0.757401                      0.390678   

            Mean harmonics-to-noise ratio  
values only                         6.064  
meta                             6.064 dB  

[2 rows x 27 columns]

### Extract feats from audio files using Praat_IdeaLogic lib 

*__in:__* directory path with extention (.wav)  
*__out:__* X, y, err_cnt, err_list, exeption_list

In [8]:
def extract(audio_file):
    name=os.path.basename(audio_file)
    name=os.path.splitext(name)[0]
    label = int(name[7:8]) - 1
    
    return label, audio_file
    
    
def return_feats(dir_name):
    X = []
    y = []
    err_cnt = 0
    err_list = []
    exeption_list = []
    
    #progression bar 
    file_count = len(dir_name)
    bar = tqdm(desc="proccessing files", total=file_count)
    tot = 0
    
    for file in dir_name:
        bar.update(1)
        tot += 1
        label, audio_file = extract(file)
        
        try:
            X.append(PLogic.get_sound_report(audio_file).to_numpy()[0])
            y.append(label)
            
        except Exception as e:
            
            err_cnt += 1
            err_list.append(audio_file)
            exeption_list.append(str(e))
            print("{}.error at:".format(err_cnt), audio_file)
            
        
    return X, y, err_cnt, err_list, exeption_list

In [4]:
# dir_name = glob.glob("Audio_Data/all_actors/*.wav")

# X, y, _, _, _ = return_feats(dir_name)
# X_name, y_name = 'vanilla_data/X.joblib', 'vanilla_data/y.joblib'

# joblib.dump(np.float64(X), X_name)
# joblib.dump(y, y_name)

1.error at: Audio_Data/all_actors/03-01-04-01-01-02-05.wav
2.error at: Audio_Data/all_actors/03-01-04-01-01-01-09.wav
3.error at: Audio_Data/all_actors/03-01-04-02-01-02-09.wav


NameError: name 'np' is not defined

In [11]:
X = joblib.load('X.joblib')
y = joblib.load('y.joblib')

In [12]:
X.shape

(5249, 27)

### Train CNN

In [6]:
from sklearn.model_selection import train_test_split
import keras
import numpy as np
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [7]:
# # model 1
# model = Sequential()

# model.add(Conv1D(64, 5, padding='same',
#                          input_shape=(1, 27)))
# model.add(Activation('relu'))
# model.add(Dropout(0.2))
# model.add(Flatten())
# model.add(Dense(8))
# model.add(Activation('softmax'))

# model.summary()
# # acc ~= 44

In [8]:
# model 2
model = Sequential()

model.add(Conv1D(128, 5,padding='same',
                 input_shape=(1, 27)))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(8), padding='same'))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(8))
model.add(Activation('softmax'))

# model.summary()

In [9]:
opt = keras.optimizers.RMSprop(lr=0.00005, rho=0.9, epsilon=0.1, decay=0.0)

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [10]:
X_train = np.array(X_train)
X_train = np.expand_dims(X_train, axis=1)
X_test = np.array(X_test)
X_test = np.expand_dims(X_test, axis=1)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [11]:
history = model.fit(X_train, y_train, batch_size=16, epochs=1000, validation_data=(X_test, y_test))

Train on 3516 samples, validate on 1733 samples
Epoch 1/1000
3516/3516 [==============================] - 5s 2ms/step - loss: 6.8079 - accuracy: 0.1459 - val_loss: 2.2396 - val_accuracy: 0.1685
Epoch 2/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 4.2495 - accuracy: 0.1800 - val_loss: 1.8709 - val_accuracy: 0.2839
Epoch 3/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 3.0422 - accuracy: 0.2267 - val_loss: 1.6994 - val_accuracy: 0.3808
Epoch 4/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 2.4832 - accuracy: 0.2341 - val_loss: 1.6702 - val_accuracy: 0.3491
Epoch 5/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 2.1946 - accuracy: 0.2489 - val_loss: 1.6906 - val_accuracy: 0.3266
Epoch 6/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 2.0285 - accuracy: 0.2654 - val_loss: 1.6844 - val_accuracy: 0.3295
Epoch 7/1000
3516/3516 [==============================] - 4s 1ms/step - loss

Epoch 112/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 1.0361 - accuracy: 0.6257 - val_loss: 0.9777 - val_accuracy: 0.6497
Epoch 113/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 1.0567 - accuracy: 0.6086 - val_loss: 0.9828 - val_accuracy: 0.6457
Epoch 114/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 1.0532 - accuracy: 0.6109 - val_loss: 0.9777 - val_accuracy: 0.6532
Epoch 115/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 1.0349 - accuracy: 0.6249 - val_loss: 0.9748 - val_accuracy: 0.6520
Epoch 116/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 1.0438 - accuracy: 0.6158 - val_loss: 0.9685 - val_accuracy: 0.6538
Epoch 117/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 1.0328 - accuracy: 0.6163 - val_loss: 0.9750 - val_accuracy: 0.6394
Epoch 118/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 1.0332 - accuracy: 0.6166 - val_

3516/3516 [==============================] - 4s 1ms/step - loss: 0.8560 - accuracy: 0.6940 - val_loss: 0.8195 - val_accuracy: 0.7201
Epoch 223/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.8677 - accuracy: 0.6815 - val_loss: 0.8183 - val_accuracy: 0.7178
Epoch 224/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.8622 - accuracy: 0.6837 - val_loss: 0.8209 - val_accuracy: 0.7138
Epoch 225/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.8661 - accuracy: 0.6846 - val_loss: 0.8247 - val_accuracy: 0.7167
Epoch 226/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.8591 - accuracy: 0.6908 - val_loss: 0.8183 - val_accuracy: 0.7109
Epoch 227/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.8617 - accuracy: 0.6840 - val_loss: 0.8135 - val_accuracy: 0.7207
Epoch 228/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.8505 - accuracy: 0.6877 - val_loss: 0.8146 - 

3516/3516 [==============================] - 4s 1ms/step - loss: 0.7653 - accuracy: 0.7216 - val_loss: 0.7549 - val_accuracy: 0.7392
Epoch 333/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.7491 - accuracy: 0.7224 - val_loss: 0.7483 - val_accuracy: 0.7357
Epoch 334/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.7511 - accuracy: 0.7244 - val_loss: 0.7460 - val_accuracy: 0.7363
Epoch 335/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.7634 - accuracy: 0.7227 - val_loss: 0.7489 - val_accuracy: 0.7357
Epoch 336/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.7797 - accuracy: 0.7088 - val_loss: 0.7478 - val_accuracy: 0.7276
Epoch 337/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.7506 - accuracy: 0.7247 - val_loss: 0.7440 - val_accuracy: 0.7346
Epoch 338/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.7676 - accuracy: 0.7233 - val_loss: 0.7418 - 

3516/3516 [==============================] - 4s 1ms/step - loss: 0.7054 - accuracy: 0.7423 - val_loss: 0.6976 - val_accuracy: 0.7565
Epoch 443/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.7040 - accuracy: 0.7389 - val_loss: 0.7047 - val_accuracy: 0.7553
Epoch 444/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.7007 - accuracy: 0.7489 - val_loss: 0.7139 - val_accuracy: 0.7403
Epoch 445/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.6967 - accuracy: 0.7386 - val_loss: 0.7015 - val_accuracy: 0.7565
Epoch 446/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.6911 - accuracy: 0.7460 - val_loss: 0.7036 - val_accuracy: 0.7513
Epoch 447/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.6970 - accuracy: 0.7412 - val_loss: 0.7030 - val_accuracy: 0.7576
Epoch 448/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.6973 - accuracy: 0.7437 - val_loss: 0.6952 - 

3516/3516 [==============================] - 4s 1ms/step - loss: 0.6513 - accuracy: 0.7634 - val_loss: 0.6835 - val_accuracy: 0.7571
Epoch 553/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.6532 - accuracy: 0.7659 - val_loss: 0.6696 - val_accuracy: 0.7623
Epoch 554/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.6558 - accuracy: 0.7560 - val_loss: 0.6705 - val_accuracy: 0.7611
Epoch 555/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.6443 - accuracy: 0.7628 - val_loss: 0.6703 - val_accuracy: 0.7594
Epoch 556/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.6550 - accuracy: 0.7543 - val_loss: 0.6902 - val_accuracy: 0.7519
Epoch 557/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.6610 - accuracy: 0.7546 - val_loss: 0.6752 - val_accuracy: 0.7548
Epoch 558/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.6519 - accuracy: 0.7560 - val_loss: 0.6699 - 

3516/3516 [==============================] - 4s 1ms/step - loss: 0.6196 - accuracy: 0.7676 - val_loss: 0.6539 - val_accuracy: 0.7663
Epoch 663/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.6154 - accuracy: 0.7756 - val_loss: 0.6632 - val_accuracy: 0.7657
Epoch 664/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.6134 - accuracy: 0.7710 - val_loss: 0.6506 - val_accuracy: 0.7709
Epoch 665/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.6252 - accuracy: 0.7665 - val_loss: 0.6499 - val_accuracy: 0.7767
Epoch 666/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.6260 - accuracy: 0.7676 - val_loss: 0.6591 - val_accuracy: 0.7675
Epoch 667/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.6219 - accuracy: 0.7639 - val_loss: 0.6627 - val_accuracy: 0.7651
Epoch 668/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.6335 - accuracy: 0.7608 - val_loss: 0.6570 - 

3516/3516 [==============================] - 4s 1ms/step - loss: 0.6075 - accuracy: 0.7742 - val_loss: 0.6360 - val_accuracy: 0.7807
Epoch 773/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.6005 - accuracy: 0.7776 - val_loss: 0.6447 - val_accuracy: 0.7721
Epoch 774/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.5964 - accuracy: 0.7745 - val_loss: 0.6431 - val_accuracy: 0.7692
Epoch 775/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.5859 - accuracy: 0.7847 - val_loss: 0.6440 - val_accuracy: 0.7750
Epoch 776/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.5928 - accuracy: 0.7759 - val_loss: 0.6468 - val_accuracy: 0.7726
Epoch 777/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.5931 - accuracy: 0.7793 - val_loss: 0.6396 - val_accuracy: 0.7744
Epoch 778/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.5931 - accuracy: 0.7807 - val_loss: 0.6403 - 

3516/3516 [==============================] - 4s 1ms/step - loss: 0.5600 - accuracy: 0.7895 - val_loss: 0.6323 - val_accuracy: 0.7807
Epoch 883/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.5628 - accuracy: 0.7890 - val_loss: 0.6326 - val_accuracy: 0.7778
Epoch 884/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.5746 - accuracy: 0.7873 - val_loss: 0.6404 - val_accuracy: 0.7773
Epoch 885/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.5764 - accuracy: 0.7824 - val_loss: 0.6423 - val_accuracy: 0.7709
Epoch 886/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.5696 - accuracy: 0.7853 - val_loss: 0.6309 - val_accuracy: 0.7790
Epoch 887/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.5862 - accuracy: 0.7796 - val_loss: 0.6322 - val_accuracy: 0.7836
Epoch 888/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.5672 - accuracy: 0.7878 - val_loss: 0.6420 - 

3516/3516 [==============================] - 4s 1ms/step - loss: 0.5591 - accuracy: 0.7995 - val_loss: 0.6220 - val_accuracy: 0.7819
Epoch 993/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.5584 - accuracy: 0.7924 - val_loss: 0.6328 - val_accuracy: 0.7761
Epoch 994/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.5584 - accuracy: 0.7947 - val_loss: 0.6239 - val_accuracy: 0.7778
Epoch 995/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.5580 - accuracy: 0.7921 - val_loss: 0.6297 - val_accuracy: 0.7825
Epoch 996/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.5582 - accuracy: 0.7892 - val_loss: 0.6251 - val_accuracy: 0.7807
Epoch 997/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.5499 - accuracy: 0.7989 - val_loss: 0.6266 - val_accuracy: 0.7750
Epoch 998/1000
3516/3516 [==============================] - 4s 1ms/step - loss: 0.5533 - accuracy: 0.7915 - val_loss: 0.6372 - 

In [12]:
model.evaluate(X_test, y_test)

1733/1733 [==============================] - 0s 227us/step


[0.6296181382640413, 0.7761107683181763]